## **Simulações numéricas 3D em GPUs de hipertermia com nanopartículas por um modelo de bioaquecimento não linear**

Nome: Lucas Martins Oliveira

Equação a ser modelada:

\begin{equation*}
  \begin{cases}
    \displaystyle \nabla \cdot k \nabla T + w_b c_b(T_a - T) + Q_m = 0 \text{ para } \Omega\\\\
    \displaystyle k \nabla T \cdot\vec{n} = 0 \text{ para } \partial \Omega_N\\\\
    \displaystyle T  = 37 \text{ para } \partial \Omega_D\\\\
    \displaystyle T(.,0) = 37
  \end{cases}
\end{equation*}

\begin{equation*}
  \text { onde } T : \Omega \times  I \rightarrow \mathbb{R^+} \text { representa o campo de temperatura do tecido } \\
  c_b : \Omega \times  I \rightarrow \mathbb{R^+} \text { representa o calor específico do sangue } \\
  k : \Omega \times  I \rightarrow \mathbb{R^+} \text { na qual a condutividade térmica é considerada isotrópica, mas não homogênea } \\
  w_b:  \Omega \times {R^+} \times  I \rightarrow \mathbb{R^+} \text { taxa de perfusão sanguínea } \\
  T_a \text { temperatura arterial } \\
  Q_m : \Omega \times  I \rightarrow \mathbb{R^+} \text { representa a geração de calor metabólico } \\
  f : \Omega \times  I \rightarrow \mathbb{R^+} \text { temperaturaprescrita(Dirichelet) ou fluxo (Neumann ou Robin} \\
  \alpha , \beta : \Omega \times  I \rightarrow \mathbb{R^+} \text { sendo o vetor normal unitário } \\
\end{equation*}

Inicialmente definimos o dominio, em questão

In [ ]:
%%writefile dominio.h

#ifndef DOMINIO_H
#define DOMINIO_H

class Dominio {
  private:
    int tamX, tamY;
    float h;
    float *x;
    float *y;

  public:
    Dominio(float passo, int tamx, int tamy);
    ~Dominio();
    void setX();
    void setY();
    float getX(int indice);
    float getY(int indice);
};

#endif //DOMINIO_H

Writing dominio.h


In [ ]:
%%writefile dominio.cpp

#include <iostream>
using namespace std;

#include "dominio.h"

Dominio::Dominio(float passo, int tamx, int tamy) {
  h = passo;
  tamX = tamx;
  tamY = tamy;
  x = new float[tamX];
  y = new float[tamY];
}

Dominio::~Dominio() {
  delete [] x;
  delete [] y;
}

void Dominio::setX() {
  for(int i=0; i<tamX; i++) {
      x[i] = i*h;
  }
}

void Dominio::setY() {
  for(int i=0; i<tamY; i++) {
      y[i] = i*h;
  }
}

float Dominio::getX(int indice) {
  if(indice < tamX)
  {
    return x[indice];
  }
  else
  {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

float Dominio::getY(int indice) {
  if(indice < tamY)
  {
    return y[indice];
  }
  else
  {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

Overwriting dominio.cpp


Assim modelamos agora as regiões do nosso espaço em questão que é divida em 4:


---



0.   Derme
1.   Gordura
2.   Músculo
3.   Tumor



---





In [ ]:
%%writefile tecidos.h

#ifndef TECIDOS_H
#define TECIDOS_H

class Tecidos {
  private:
    int *tecidosHumanos;
    int tamX;
    int tamY;
    int detInd(int i, int j);

  public:
    Tecidos(int tamx, int tamy);
    ~Tecidos();
    void set();
    int busca(int i, int j);
    void salvaMatriz();
};

#endif //TECIDOS_H

Overwriting tecidos.h


In [ ]:
%%writefile tecidos.cpp

#include <iostream>
using namespace std;

#include <fstream>
#include <vector>

#include "tecidos.h"

Tecidos::Tecidos(int tamx, int tamy) {
  tamX = tamx;
  tamY = tamy;
  tecidosHumanos = new int[tamX*tamY];
}

Tecidos::~Tecidos() {
  delete [] tecidosHumanos;
}

int Tecidos::detInd(int i, int j) {
  if(i<tamX && j<tamY)
  {
    return tamY*i + j;
  }
  else
  {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

void Tecidos::set() {
  for(int i=0; i<tamX; i++) {
     for(int j=0; j<tamY; j++) {
        if((i>(19) && i<(45) && j>(37) && j<(63)))
        {
          //tumor esfera de raio 13mm -> consideramos um quadrado nessa abordagem
          //centro (20,50)mm
          tecidosHumanos[detInd(i,j)] = 3;
        }
        else if(i<2)
        {
          //derme 2mm
          tecidosHumanos[detInd(i,j)] = 0;
        }
        else if(i<12)
        {
          //adiposo
          tecidosHumanos[detInd(i,j)] = 1;
        }
        else
        {
          //musculo
          tecidosHumanos[detInd(i,j)] = 2;
        }
     }
  }
}

int Tecidos::busca(int i, int j) {
  if(i < tamX && j < tamY)
  {
    return tecidosHumanos[detInd(i,j)];
  }
  else
  {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

void Tecidos::salvaMatriz() {
    // Matriz que vai receber os valores dos tecidos
    vector<vector<int>> matriz;

    // Inicializar a matriz que vai ser salva com elementos
    matriz.resize(tamX);
    for (int i = 0; i < tamX; i++) {
      matriz[i].resize(tamY);
      for (int j = 0; j < tamY; j++) {
        matriz[i][j] =  busca(i,j);
      }
    }

    // Salvar a matriz de tecidos em um arquivo para ser importada posteriormente
    ofstream arquivo("matriz.txt");
    if (arquivo.is_open()) {
      for (const auto& linha : matriz) {
        for (const auto& elemento : linha) {
          arquivo << elemento << " ";
        }
        arquivo << "\n";
      }
      arquivo.close();
      cout << "C++: Matriz de tecidos salva em matriz.txt" << endl;
    } else {
      cerr << "C++: Erro ao abrir o arquivo" << endl;
    }
}

Overwriting tecidos.cpp


A geração de calor metabólico varia de acordo com a região tratada, temos assim então:

\begin{equation*}
  Q_{m_{muscle}} = Q_{m_{fat}} = Q_{m_{dermis}} = 420.0 \\
  Q_{m_{tumor}} = 4200.0
\end{equation*}

Temos que a condutividade térmica varia de acordo com o tecido, assim obtemos a seguintes relações:

\begin{equation*}
  k_{dermis} = 0.40\\
  k_{fat} = 0.21\\
  k_{muscle} = 0.45\\
  k_{tumor} = 0.55\\
\end{equation*}

A densidade dos tecidos são diferentes uma das outras, dessa forma definimos para a mesma a seguinte variação:

\begin{equation*}
  \rho_{dermis} = 1200\\
  \rho_{fat} = 1000\\
  \rho_{muscle} = 1000\\
  \rho_{tumor} = 1000\\
\end{equation*}

O calor específico dos tecidos são diferentes um dos outros, dessa forma definimos para a mesma a seguinte variação:

\begin{equation*}
  c_{dermis} = 3600\\
  c_{fat} = 2500\\
  c_{muscle} = 3800\\
  c_{tumor} = 4200\\
\end{equation*}

A taxa de perfusão sanguínea varia de acordo com o tecido e com a temperatura que esse se encontra, dessa forma temos que:

\begin{equation*}
  W_{dermis} = 0.5 \\
  W_{fat} =
    \begin{cases}
      \displaystyle 0.36 + 0.36exp(-\frac{(T-45.0)^2}{12.0}) \text { para } T \leq 45.0 \\\\
      0.72 \text { para } T>45.0
    \end{cases} \\
  W_{muscle} =
    \begin{cases}
      \displaystyle 0.45 + 3.55exp(-\frac{(T-45.0)^2}{12.0}) \text { para } T \leq 45.0 \\\\
      4.0 \text { para } T>45.0
    \end{cases} \\
    W_{tumor} =
    \begin{cases}
      \displaystyle 0.833 -\frac{(T-37.0)^{4.8}}{5.438E+3} \text { para } 37.0 \leq T \leq 42.0 \\\\
      0.416 \text { para } T>42.0
    \end{cases} \\
\end{equation*}

In [ ]:
%%writefile variaveis.h

#ifndef VARIAVEIS_H
#define VARIAVEIS_H

class Variaveis {
  private:
   float *qm;
   float *k;
   float *w;
   float *p;
   float *c;

  public:
    Variaveis();
    ~Variaveis();
    void setQm();
    void setK();
    void setP();
    void setC();
    void setW();
    float getQm(int tecido);
    float* getK();
    float getP(int tecido);
    float getC(int tecido);
    float getW(int tecido, float temp);
};

#endif //VARIAVEIS_H

Overwriting variaveis.h


In [ ]:
%%writefile variaveis.cpp

#include <iostream>
#include <cmath>
using namespace std;

#include "variaveis.h"

Variaveis::Variaveis() {
  qm = new float[4];
  k = new float[4];
  w = new float[4];
  p = new float[4];
  c = new float[4];
}

Variaveis::~Variaveis() {
  delete [] qm;
  delete [] k;
  delete [] w;
  delete [] p;
  delete [] c;
}

void Variaveis::setQm() {
  for(int i=0; i<4; i++) {
    i==3 ? qm[i]=4200 : qm[i]=420;
  }
}

void Variaveis::setK() {
  k[0] = 0.4;
  k[1] = 0.21;
  k[2] = 0.45;
  k[3] = 0.55;
}

void Variaveis::setP() {
  for(int i=0; i<4; i++) {
    i==0 ? p[i]=1200 : p[i]=1000;
  }
}

void Variaveis::setC() {
  c[0] = 3600;
  c[1] = 2500;
  c[2] = 3800;
  c[3] = 4200;
}

void Variaveis::setW() {
  w[0] = 0.5;
  w[1] = 4.0;
  w[2] = 0.72;
  w[3] = 0.416;
}

float Variaveis::getQm(int tecido) {
  if(tecido <= 3 && tecido >=0)
    return qm[tecido];
  else {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

float* Variaveis::getK() {
  return k;
}

float Variaveis::getP(int tecido) {
  if(tecido <= 3 && tecido >=0)
    return p[tecido];
  else {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

float Variaveis::getC(int tecido) {
  if(tecido <= 3 && tecido >=0)
    return c[tecido];
  else {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

float Variaveis::getW(int tecido, float temp) {
  if(tecido == 0)
    return w[0];
  else if(tecido == 1)
    return temp > 45.0 ? w[1] : ( 0.36 + 0.36*exp(-pow(temp - 45, 2)/12));
  else if(tecido == 2)
    return temp > 45.0 ? w[2] : ( 0.45 + 3.55*exp(-pow(temp - 45, 2)/12));
  else if(tecido == 3)
    return temp > 42.0 ? w[3] : ( 0.833 - (pow(temp-37, 4.8)/5438));
  else {
    cout << "INDICE INVÁLIDO" << endl;
    exit(1);
  }
}

Overwriting variaveis.cpp


\begin{equation*}
  \text{Temos que, reorganizando a equação separamos aonde a variavel } T_{i,j}
  \text { para o lado esquerdo da equação, e aonde não consta para o lado direito, então obtemos: }
\end{equation*}

Lado direito da equação, chamamos de f

\begin{equation*}
  f_{i,j} = -Q_{m_{i,j}} - W_{b_{i,j}}C_bT_a
\end{equation*}

Assim de forma encontramos a equação:

\begin{equation*}
  T_{i,j} = \frac{-h^2f_{i,j} + k_{i+\frac{1}{2},j}T_{i+1,j} + k_{i-\frac{1}{2},j}T_{i-1,j} + k_{i,j+\frac{1}{2}}T_{i,j+1} + k_{i,j-\frac{1}{2}}T_{i,j-1}}{k_{i+\frac{1}{2},j} + k_{i-\frac{1}{2}} + k_{i,j+\frac{1}{2}} + k_{i,j-\frac{1}{2}} + W_{b_{i,j}}C_bh^2}
\end{equation*}

Assim temos que a condutividade térmica, pode ser calculada usando o método do ponto do médio, então:
\begin{equation*}
  k_{i+\frac{1}{2},j} \approx \frac{2k_{i,j}k_{i+1,j}}{k_{i,j}+k_{i+1,j}}
\end{equation*}

In [ ]:
%%writefile jacobi.h

#ifndef JACOBI_H
#define JACOBI_H

#include <string>

#include "tecidos.h"
#include "variaveis.h"

class Jacobi {
  private:
    float **u; //vetor solucao e passo anterior ao da solucao
    float *f; //lado direito da funcao
    int tamX;
    int tamY;
    int detInd(int i, int j);

  public:
    Jacobi(int tamx, int tamy);
    ~Jacobi();
    void setU(int ci);
    void resolucao(float cb, float Ta, Tecidos *t, Variaveis *var, float h);
    void salvaMatriz(float *mat);
};

#endif //JACOBI_H

Overwriting jacobi.h


In [ ]:
%%writefile jacobi.cpp

#include <iostream>
#include <cmath>
using namespace std;
#include <string>
#include <fstream>
#include <vector>

#include "jacobi.h"
#include "tecidos.h"
#include "variaveis.h"

Jacobi::Jacobi(int tamx, int tamy) {
  tamX = tamx;
  tamY = tamy;

  f = new float[tamx*tamy];
  u = new float *[2];

  for(int i=0; i<2; i++) {
    u[i] = new float[tamX*tamY];
  }
}

Jacobi::~Jacobi() {
  for (int i = 0; i < 2; i++) {
        delete[] u[i];
  }

  delete [] f;
  delete [] u;
}

int Jacobi::detInd(int i, int j) {
   if(i<tamX && j<tamY)
  {
    return tamY*i + j;
  }
  else
  {
    cout << "INDICE INVALIDO" << endl;
    exit(1);
  }
}

void Jacobi::setU(int ci) {
  //ci == chute inicial

  for(int i=0; i<tamX; i++) {
    for(int j=0; j<tamY; j++) {
      u[0][detInd(i,j)] = 37;
    }
  }
}

void Jacobi::salvaMatriz(float *mat) {
    // Matriz que vai receber os valores da temperatura por região
    vector<vector<float>> final;

    // Inicializar a final que vai ser salva com elementos
    final.resize(tamX);
    for (int i = 0; i < tamX; i++) {
      final[i].resize(tamY);
      for (int j = 0; j < tamY; j++) {
        final[i][j] =  mat[detInd(i,j)];
      }
    }

    // Salvar a final de tecidos em um arquivo para ser importada posteriormente
    ofstream arquivo("final.txt");
    if (arquivo.is_open()) {
      for (const auto& linha : final) {
        for (const auto& elemento : linha) {
          arquivo << elemento << " ";
        }
        arquivo << "\n";
      }
      arquivo.close();
      cout << "C++: Matriz solucao salva em final.txt" << endl;
    } else {
      cerr << "C++: Erro ao abrir o arquivo" << endl;
    }
}

void Jacobi::resolucao(float cb, float Ta, Tecidos *t, Variaveis *var, float h) {
  //Condicoes de Contorno
  float u_a = 37.0; //dirichelet esquerdo -> T0
  int u_b = 0; //neumann topo
  int u_c = 0; //neumann direito
  int u_d = 0; //neumann base

  int kit=0;
  int k_max = 10000; //numero maximo de iteraçoes do metodo
  double error= 1;
  double error_max = 1*pow(10,-6); //erro para convergencia

  float *ks = var->getK();

  bool swap = false;

  while (kit<k_max && error > error_max) { //metodo de jacob
    for(int i=0; i<tamX; i++) {
      for(int j=0; j<tamY; j++) {
         float kij = ks[t->busca(i,j)]; //vai ser usado em todos os casos

         float k_ip = i==tamX-1 ? 2*kij*kij/(2*kij) : 2*kij*(ks[t->busca(i+1,j)]) / (kij+ks[t->busca(i+1,j)]);
         float k_im = i==0 ? 2*kij*kij/(2*kij) : 2*kij*(ks[t->busca(i-1,j)]) / (kij+ks[t->busca(i-1,j)]);
         float k_jp = j==tamY-1 ? 2*kij*kij/(2*kij) : 2*kij*(ks[t->busca(i,j+1)]) / (kij+ks[t->busca(i,j+1)]);
         float k_jm = j==0 ? 2*kij*kij/(2*kij): 2*kij*(ks[t->busca(i,j-1)]) / (kij+ks[t->busca(i,j-1)]);

        //para tratar do tipo Dirichlet, devemos atribuir diretamente no ponto (x_i,y_j)
        if (i==0)
          u[!swap][detInd(i,j)] = u_a;
        else {
          //Tradando condicoes de contorno do tipo Neumann

          float uijp = j==tamY-1 ? 2*h*u_c+u[swap][detInd(i,tamY-2)] : u[swap][detInd(i,j+1)];
          float uimj = u[swap][detInd(i-1,j)];
          float uipj = i==tamX-1 ? 2*h*u_d+u[swap][detInd(tamX-2,j)] : u[swap][detInd(i+1,j)];
          float uijm = j==0 ? 2*h*u_b+u[swap][detInd(i,1)] : u[swap][detInd(i,j-1)];

          int indice = t->busca(i,j);
          int pos = detInd(i,j);

          f[pos] = -(var->getQm(indice) + var->getW(indice, u[swap][pos])*cb*Ta);
          u[!swap][pos] = (k_ip*uipj + k_im*uimj + k_jp*uijp + k_jm*uijm - f[pos]*pow(h,2)) / ( k_ip+ k_im + k_jp + k_jm + var->getW(indice, u[swap][pos])*cb*pow(h,2));
        }
      }
    }
    kit=kit+1;

    error = 1;

    for(int i=0; i<tamX; i++) {
      for(int j=0; j<tamY; j++) {
        int pos = detInd(i,j);

        error += (u[!swap][pos] - u[swap][pos]) / u[!swap][pos];
      }
    }

    swap = !swap;
  }

  cout << error << endl;

  salvaMatriz(u[swap]);
}

Overwriting jacobi.cpp


In [ ]:
%%writefile main.cpp
#include <iostream>
using namespace std;
#include <string>

#include "dominio.h"
#include "tecidos.h"
#include "variaveis.h"
#include "jacobi.h"


int main()
{
    float h = 0.001; // 0.1/(101-1);
    int tamX = 51;
    int tamY = 101;

    Dominio *dom = new Dominio(h, tamX, tamY);
    dom->setX();
    dom->setY();

    Tecidos *humano = new Tecidos(tamX, tamY);
    humano->set();
    humano->salvaMatriz();

    Variaveis *var = new Variaveis;
    var->setQm();
    var->setK();
    var->setP();
    var->setC();
    var->setW();

    int cb = 4200; //calor especifico do sangue (constante)
    int Ta = 37; //temperatura inicial (constante)

    Jacobi solucao(tamX, tamY);
    solucao.setU(Ta);
    solucao.resolucao(cb, Ta, humano, var, h);

    delete [] dom;
    delete [] humano;
    delete [] var;

    return 0;
}

Overwriting main.cpp


In [ ]:
!g++ dominio.cpp \
  tecidos.cpp \
  variaveis.cpp \
  jacobi.cpp \
  main.cpp \
  -o hyperthermia

In [ ]:
!./hyperthermia

C++: Matriz de tecidos salva em matriz.txt
1
C++: Matriz solucao salva em final.txt


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d, Axes3D
import numpy as np
import math

from google.colab import files
from matplotlib.figure import projections

plt.rcParams['figure.figsize'] = [8, 6]

# Carregar a matriz do arquivo
matriz = np.loadtxt('matriz.txt')

# Exibir a matriz
print("Matriz de tecidos:")
print(matriz)
plt.imshow(matriz)

Matriz de tecidos:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]]


In [ ]:
# Carregar a matriz do arquivo
resultados = np.loadtxt('final.txt')

# Exibir a matriz
#print(resultados)

fig = plt.figure()
plt.xlabel('x-axis (m)')
plt.ylabel('Temperatura (°C)')
plt.ylabel('z-axis (m)')
plt.imshow(np.transpose(resultados), vmin = 37.0, vmax = 37.65, cmap = 'rainbow', interpolation = 'nearest', origin = 'lower', extent=[0,0.05,0,0.1])
plt.colorbar(label = 'Temperatura (°C)')
plt.show()